In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0
from kaggle_datasets import KaggleDatasets

# Step 1: Initialize TPU
# try:
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # Detect TPU
#     tf.config.experimental_connect_to_cluster(tpu)
#     tf.tpu.experimental.initialize_tpu_system(tpu)
#     strategy = tf.distribute.TPUStrategy(tpu)
#     print("All TPU devices: ", tf.config.list_logical_devices('TPU'))
# except ValueError:
#     print("TPU not found.")

strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
# mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

# Use GPU

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


# Dataset 

In [3]:
import keras

In [4]:
image_size = (224, 224)
batch_size = 128

train_ds, val_ds = keras.utils.image_dataset_from_directory(
    "/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train",
    validation_split=0.2,
    subset="both",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

Found 100000 files belonging to 2 classes.
Using 80000 files for training.
Using 20000 files for validation.


In [5]:
with strategy.scope():
    model = tf.keras.Sequential([
        EfficientNetB0(include_top=False, input_shape=(224, 224, 3), pooling='avg'),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

# Step 4: Train Model
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    steps_per_epoch=len(train_ds) // batch_size,
    validation_steps=len(val_ds) // batch_size
)

# Step 5: Save Model
model.save("efficientnet_cifake.h5")

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/10


2025-01-19 16:40:00.292155: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/sequential_1/efficientnetb0_1/block2b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


4/4 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step - accuracy: 0.6372 - loss: 0.6142 - val_accuracy: 0.6875 - val_loss: 0.5834
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 927ms/step - accuracy: 0.8823 - loss: 0.2655 - val_accuracy: 0.6016 - val_loss: 0.8035
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 937ms/step - accuracy: 0.8766 - loss: 0.2912 - val_accuracy: 0.6484 - val_loss: 0.9052
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 924ms/step - accuracy: 0.8831 - loss: 0.2819 - val_accuracy: 0.5391 - val_loss: 1.8056
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 926ms/step - accuracy: 0.9484 - loss: 0.1687 - val_accuracy: 0.5469 - val_loss: 2.1214
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 925ms/step - accuracy: 0.9120 - loss: 0.2046 - val_accuracy: 0.5469 - val_loss: 2.0106
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 936ms/step - accuracy: 0.9065 - loss: 0.2027 - val_accuracy: 0.5391 - val_loss: 2.3171
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 938ms/step - accuracy: 0.9435 - loss: 0.1416 - val_accuracy: 0.5312 - val_loss: 3.5001
Epoch

# On Test Dataset

In [6]:
import tensorflow as tf

# Define the same preprocessing steps
def preprocess_image(image, label):
    image = tf.image.resize(image, (224, 224))  # Resize for EfficientNet
    image = image / 255.0  # Normalize to [0, 1]
    return image, label

# Load test dataset
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    '/kaggle/input/cifake-real-and-ai-generated-synthetic-images/test',
    image_size=(224, 224),  # Resize for EfficientNet
    batch_size=32,
    shuffle=False  # Do not shuffle for consistent evaluation
)

# Apply preprocessing
test_dataset = test_dataset.map(preprocess_image)


Found 20000 files belonging to 2 classes.


In [7]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.2f}%")

625/625 ━━━━━━━━━━━━━━━━━━━━ 33s 52ms/step - accuracy: 0.8456 - loss: 1.2708
Test Loss: 4.1154
Test Accuracy: 0.50%
